## Theory

One of the most powerful tools in Deep Learning is that sometimes we can take the knowledge or parameters the neural network has learned from one task and apply that knowledge to a different task. So for example maybe we have a neural network model, learned to recognize objects like cats, dogs, and other animals. Then we use that knowledge or use a part of it to do a better job at reading X-ray scans. This is called Transfer Learning. To have a more concrete definition, in transfer learning we reuse a pre-trained model on a new problem. This is particularly so useful because in Deep learning we can train more complex models, with fewer quantities of data using this method. This might come in handy in Data Science because, in most real-world problems, there’s a lack of labeled data points to train such complex models.

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

C:\Users\Krishna\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Krishna\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\Krishna\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\Krishna\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


Let's train a neural network on Fashion MNIST:

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=15,
                    validation_data=(X_valid, y_valid), verbose=2)

In [ ]:
## Saving the model

model.save("full_mnist_model.h5")

In [ ]:
load_full_mnist_model = tf.keras.models.load_model("full_mnist_model.h5")

In [ ]:
load_full_mnist_model.summary()

In [ ]:
np.where(y_train % 2 == 0, 1 , 0)

In [ ]:
y_train[0], y_train[-1] 

In [ ]:
def update_even_odd_labels(labels):
    for idx, label in enumerate(labels):
        labels[idx] = np.where(label % 2 == 0, 1 , 0)
    return labels

In [ ]:
y_train_bin, y_valid_bin, y_test_bin = update_even_odd_labels([y_train, y_valid, y_test])

In [ ]:
# Check trainable or not:

for layer in load_full_mnist_model.layers:
    print(f"{layer.name}: {layer.trainable}")

In [ ]:
# make lower layer UN-trainable except the last layer:

for layer in load_full_mnist_model.layers[:-1]:
    layer.trainable = False
    print(f"Now {layer.name}: {layer.trainable}")

In [ ]:
new_model = tf.keras.models.Sequential(load_full_mnist_model.layers[:-1])
new_model.add(
    tf.keras.layers.Dense(1, activation="sigmoid")
)

In [ ]:
new_model.summary()

In [ ]:
new_model.get_config()

In [ ]:
# Check trainable or not:

for layer in new_model.layers:
    print(f"{layer.name}: {layer.trainable}")

In [ ]:
new_model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [ ]:
new_history = new_model.fit(X_train, y_train_bin, epochs=15,
                    validation_data=(X_valid, y_valid_bin), verbose=2)

In [ ]:
new_model.evaluate(X_test, y_test_bin)

In [ ]:
X_new = X_test[:3]
y_test[:3], y_test_bin[:3]

In [ ]:
new_model.predict(X_new)